In [1]:
#Pre processing of the data
#Loading the dataset
import pandas as pd
df=pd.read_csv("D:/UOFG/SEM2/IV/Coursework/SystemA/imdb1000moviesinjune2022.csv")
df['Year'] = df['Year'].str.replace('-', '')
df['Year'] = df['Year'].str.replace('(', '')
df['Year'] = df['Year'].str.replace(')', '')
df['Year'] = df['Year'].str.replace('I', '')
df['GROSS'] = df['GROSS'].str.replace('M', '')
df['GROSS'] = df['GROSS'].str.replace('$', '')
df.dropna(inplace=True)
df['genre_new'] = df['genre'].str.split(',').str[0]
df['Year'] = df['Year'].astype(int)
df['GROSS'] = (df['GROSS'].astype(float)
                             .round(0)
                             .astype(int))
def categorize_year(Year):
    if Year >= 1900:
        decade_start = (Year // 10) * 10
        decade_end = decade_start + 9
        return f"{decade_start}-{decade_end}"
    else:
        return "Before 1900"

df['Decade'] = df['Year'].apply(categorize_year)
df[['Year', 'Decade']].head()

df.to_csv('D:/UOFG/SEM2/IV/Coursework/SystemA/modified_data.csv', index=False)

C:\Users\abhij\AppData\Local\Temp\ipykernel_9728\1333697793.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Year'] = df['Year'].str.replace('(', '')
C:\Users\abhij\AppData\Local\Temp\ipykernel_9728\1333697793.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Year'] = df['Year'].str.replace(')', '')
C:\Users\abhij\AppData\Local\Temp\ipykernel_9728\1333697793.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['GROSS'] = df['GROSS'].str.replace('$', '')


In [2]:
#System A

import altair as alt
import pandas as pd

# Load the dataset
df = pd.read_csv('D:/UOFG/SEM2/IV/Coursework/SystemA/modified_Data.csv')
df['Decade'] = df['Decade'].astype(str)


colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080', '#ffffff', '#000000']
color_dict = {genre: color for genre, color in zip(df['genre_new'].unique(), colors)}


color_encoding = alt.Color('genre_new:N', scale=alt.Scale(domain=list(color_dict.keys()), range=list(color_dict.values())))


selection_genre = alt.selection_multi(fields=['genre_new'], name="Genre")
selection_certificate = alt.selection_multi(fields=['certificate'], name="Certificate")
selection_decade = alt.selection_interval(encodings=['x'], name="Decade") 

donut_chart = alt.Chart(df).mark_arc(innerRadius=100).encode(
    theta=alt.Theta(field='GROSS', aggregate='average', type='quantitative'),
    color=alt.condition(selection_genre, color_encoding, alt.value('lightgray')),
    tooltip=[alt.Tooltip('genre_new', title='Genre'), alt.Tooltip('average(GROSS)', title='Average Gross(in million)', format=',.2f')]
).transform_filter(
    selection_decade  
).properties(
    title='Average Gross(in million) by Genre',
    width=400,
    height=400
).add_selection(selection_genre)

# Create the line chart with color scheme and interaction
chart2 = alt.Chart(df).mark_line(point=True).encode(
    x=alt.X('Decade:N', title='Decade'),
    y=alt.Y('average(metascore):Q', title='Average Metascore'),
    color=alt.condition(selection_genre, color_encoding, alt.value('lightgray')), 
    tooltip=['genre_new:N', 'Decade:N', 'average(metascore):Q']
).transform_filter(
    selection_genre
).properties(
    title="Average metascore by Genre over time",
    width=700,
    height=240
).add_selection(
    selection_genre,
    selection_decade 
)


color_dict_cert = {cert: color for cert, color in zip(df['certificate'].unique(), colors)}
color_encoding_cert = alt.Color('certificate:N', scale=alt.Scale(domain=list(color_dict_cert.keys()), range=list(color_dict_cert.values())))
chart3 = alt.Chart(df).mark_bar().encode(
    x=alt.X('certificate', title='Certificate'),
    y=alt.Y('sum(GROSS)', title='Total Gross(in million)', axis=alt.Axis(format=',.2f')),
    color=alt.condition(selection_certificate, color_encoding_cert, alt.value('lightgray')),  # Change color encoding here
    tooltip=[alt.Tooltip('certificate', title='Certificate'), alt.Tooltip('sum(GROSS)', title='Total Gross(in million)', format=',.2f')]
).properties(
    title="Gross earnings based on movie certificate",
    width=300,
    height=200
).add_selection(selection_certificate)

left_charts = alt.vconcat(donut_chart, chart3).resolve_scale(color='independent')
final_chart = alt.hconcat(left_charts, chart2).resolve_scale(y='independent')
final_chart.save('D:/UOFG/SEM2/IV/Coursework/SystemA/sysA.html')


D:\Anaconda\lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
D:\Anaconda\lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
